# Assignment 2

### Part A Baseline model (Frozen embeddings)

Now after having both created the balanced dataset and encoded the claims using the model so we now have the embeddings we can actually finalize Part A where we train a baseline classifier and evaluate it.

First we will load in the dataset from the parquet file and the pt files for the embeddings:

In [1]:
# Load parquet file
import pandas as pd
file_path = '/Users/anderssonderby/PythonBDS/8th_semester/Applied_Deep_Learning_and_AI/M4_assignments/patents_50k_green.parquet'

df = pd.read_parquet(file_path)

In [2]:
print(df.head())
print(df['is_green_silver'].value_counts())

        id        date                                               text  \
0  8929789  2015-01-06  1. A fixing device comprising: a rotatable, en...   
1  9253996  2016-02-09  1. A method of recovering pectin from citrus p...   
2  9580824  2017-02-28  1. An anion-conducting polymeric membrane comp...   
3  9383068  2016-07-05  1. An LED based lighting system, comprising: a...   
4  8915542  2014-12-23  1. A sunroof apparatus comprising: a movable r...   

   Y02A  Y02B  Y02C  Y02D  Y02E  Y02P  Y02T  Y02W  is_green_silver  
0     0     0     0     0     0     0     0     0                0  
1     0     0     0     0     1     1     0     0                1  
2     0     0     0     0     1     0     0     0                1  
3     0     1     0     0     0     0     0     0                1  
4     0     0     0     0     0     0     0     0                0  
is_green_silver
0    25000
1    25000
Name: count, dtype: int64


In [3]:
# Shuffle
df_shuffled = df.sample(frac=1, random_state=42)

# Split
train_silver = df_shuffled.iloc[:40000]
eval_silver = df_shuffled.iloc[40000:45000]
pool_unlabeled = df_shuffled.iloc[45000:]

print(f"Train: {len(train_silver)}, Eval: {len(eval_silver)}, Pool: {len(pool_unlabeled)}")


Train: 40000, Eval: 5000, Pool: 5000


In [4]:
import torch

# Load the embeddings (these are lists of tensors)
train_silver_embeddings = torch.load('/Users/anderssonderby/PythonBDS/8th_semester/Applied_Deep_Learning_and_AI/M4_assignments/train_silver_embeddings.pt', map_location='cpu')
eval_silver_embeddings = torch.load('/Users/anderssonderby/PythonBDS/8th_semester/Applied_Deep_Learning_and_AI/M4_assignments/eval_silver_embeddings.pt', map_location='cpu')
pool_unlabeled_embeddings = torch.load('/Users/anderssonderby/PythonBDS/8th_semester/Applied_Deep_Learning_and_AI/M4_assignments/pool_unlabeled_embeddings.pt', map_location='cpu')

print(f"Train embeddings loaded: {len(train_silver_embeddings)} items")
print(f"Eval embeddings loaded: {len(eval_silver_embeddings)} items")
print(f"Pool embeddings loaded: {len(pool_unlabeled_embeddings)} items")

Train embeddings loaded: 40000 items
Eval embeddings loaded: 5000 items
Pool embeddings loaded: 5000 items


In [5]:
# Convert tensor embeddings to numpy arrays and add to dataframes
train_silver['embeddings'] = [emb.numpy() for emb in train_silver_embeddings]
eval_silver['embeddings'] = [emb.numpy() for emb in eval_silver_embeddings]
pool_unlabeled['embeddings'] = [emb.numpy() for emb in pool_unlabeled_embeddings]

print("\nEmbeddings added to DataFrames!")
print(f"Train embedding shape: {train_silver['embeddings'].iloc[0].shape}")
print(f"Eval embedding shape: {eval_silver['embeddings'].iloc[0].shape}")
print(f"Pool embedding shape: {pool_unlabeled['embeddings'].iloc[0].shape}")


Embeddings added to DataFrames!
Train embedding shape: (768,)
Eval embedding shape: (768,)
Pool embedding shape: (768,)


In [6]:
# Verify the counts match
assert len(train_silver) == len(train_silver_embeddings), "Train size mismatch!"
assert len(eval_silver) == len(eval_silver_embeddings), "Eval size mismatch!"
assert len(pool_unlabeled) == len(pool_unlabeled_embeddings), "Pool size mismatch!"

print("\n✓ All embeddings aligned correctly with dataframes!")
print(train_silver.head())


✓ All embeddings aligned correctly with dataframes!
            id        date                                               text  \
33553  9066432  2015-06-23  1. A copper foil for a printed wiring board, c...   
9427   9416246  2016-08-16  1. A vapor phase process for producing a subst...   
199    8865275  2014-10-21  1. An optical absorption anisotropic film, whe...   
12447  9466436  2016-10-11  1. A power storage device comprising: a positi...   
39489  8841803  2014-09-23  1. A superconducting rotating machine having a...   

       Y02A  Y02B  Y02C  Y02D  Y02E  Y02P  Y02T  Y02W  is_green_silver  \
33553     0     0     0     0     0     0     0     0                0   
9427      0     0     0     0     0     0     0     0                0   
199       0     0     0     0     0     0     0     0                0   
12447     0     0     0     0     1     0     0     0                1   
39489     0     0     0     0     1     0     0     0                1   

               

#### Baseline classifier

We will use the embeddings actually train a classifier that can classify the claims into green and non-green technology. First we will prepare the data:

In [8]:
import numpy as np
# Prepare data for the classifier
X_train = np.array(train_silver['embeddings'].tolist())
y_train = train_silver['is_green_silver'].values

X_eval = np.array(eval_silver['embeddings'].tolist())
y_eval = eval_silver['is_green_silver'].values

print(f"Train embeddings shape: {X_train.shape}")
print(f"Train labels shape: {y_train.shape}")
print(f"Eval embeddings shape: {X_eval.shape}")
print(f"Eval labels shape: {y_eval.shape}")

Train embeddings shape: (40000, 768)
Train labels shape: (40000,)
Eval embeddings shape: (5000, 768)
Eval labels shape: (5000,)


We use the Logistic regression classifier which essentially learns the relationsship between the 764D embeddings space and the probability of a claim being a green technology. It is a linear model, meaning that it assumes a linear relationsship between the log-odds of the technology being green and the embedding dimensions. It is trained on the train_silver split which contains 40,000 claims and then we evaluate on the eval_silver with 5,000 claims.

In [9]:
# Initialize and train a Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
print("Training Logistic Regression classifier...")
classifier = LogisticRegression(random_state=42, solver='liblinear', n_jobs=-1)
classifier.fit(X_train, y_train)
print("Classifier training complete.")

Training Logistic Regression classifier...


/Users/anderssonderby/PythonBDS/8th_semester/Applied_Deep_Learning_and_AI/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Classifier training complete.


In [10]:
# Make predictions on the eval_silver set
y_pred = classifier.predict(X_eval)


In [11]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.80      0.79      2469
           1       0.80      0.77      0.78      2531

    accuracy                           0.79      5000
   macro avg       0.79      0.79      0.79      5000
weighted avg       0.79      0.79      0.79      5000



Based on the classification report we can see that the classifier does alright. With precision, recall and f1-score being at around 0.77-0.80 for both the green and non-green class. It has an overall accuracy of 0.79, which means that it is able to classify the claim correctly 79% of the time.

## Part B: Identify High-Risk examples (Uncertainty Sampling)

Tasks:
* Compute p_green for all examples in pool_unlabeled
* Compute u for each example and select the 100 highest-u rows
* Export hitl_green_100.csv including: doc_id, text, p_green, u, and empty labeling columns.

#### Task
Identify the top 100 high-risk examples from the `pool_unlabeled` DataFrame using uncertainty sampling. This involves predicting the probability of being 'green' (`p_green`) for each example, calculating an uncertainty score `u = 1 - 2 * |p_green - 0.5|`, and selecting the 100 examples with the highest 'u' scores. Finally, export these 100 examples to a CSV file named `hitl_green_100.csv`, including columns for `doc_id` (renamed from `id`), `text`, `p_green`, `u`, and three empty columns named 'label_1', 'label_2', and 'label_3'.

#### Compute p_green for pool_unlabeled

##### Subtask:
Generate code to predict probabilities for the 'pool_unlabeled' dataset using the trained classifier and store 'p_green' (probability of being green) in a new column in the `pool_unlabeled` DataFrame. The 'pool_unlabeled_embeddings' from the previous steps will be used.


**Reasoning**:
To predict probabilities for the 'pool_unlabeled' dataset, I will first extract the embeddings from the DataFrame, convert them to a NumPy array, then use the trained classifier to predict the probability for each class, specifically extracting the probability for the 'green' class (class 1) and storing it in a new column in the `pool_unlabeled` DataFrame.



In [12]:
X_pool = np.array(pool_unlabeled['embeddings'].tolist())

# Predict probabilities for the pool_unlabeled set
# The output will be an array where each row is [probability of class 0, probability of class 1]
probabilities = classifier.predict_proba(X_pool)

# p_green is the probability of class 1 (index 1)
p_green = probabilities[:, 1]

# Add p_green to the pool_unlabeled DataFrame
pool_unlabeled = pool_unlabeled.copy()
pool_unlabeled['p_green'] = p_green

print("Probabilities calculated and 'p_green' column added to pool_unlabeled DataFrame.")
print(pool_unlabeled[['id', 'p_green']].head())

Probabilities calculated and 'p_green' column added to pool_unlabeled DataFrame.
            id   p_green
5741   9681260  0.220310
33418  9568802  0.394104
22423  9344033  0.502703
11094  9059358  0.964665
18642  8733459  0.889267


**Reasoning**:
Now that 'p_green' is calculated, I will compute the uncertainty score 'u' for each example in `pool_unlabeled` using the formula `u = 1 - 2 * |p_green - 0.5|`. Then, I will select the top 100 examples with the highest 'u' scores.



In [13]:
# Calculate uncertainty score 'u'
pool_unlabeled['u'] = 1 - 2 * np.abs(pool_unlabeled['p_green'] - 0.5)

# Sort by 'u' in descending order and select the top 100 examples
high_risk_examples = pool_unlabeled.sort_values(by='u', ascending=False).head(100)

print("Uncertainty score 'u' calculated and top 100 high-risk examples selected.")
print(high_risk_examples[['id', 'p_green', 'u']].head())

Uncertainty score 'u' calculated and top 100 high-risk examples selected.
            id   p_green         u
39200  9447102  0.499943  0.999886
42324  8607927  0.499938  0.999876
13598  8984809  0.499823  0.999647
48261  8809354  0.500248  0.999504
11824  9227385  0.499463  0.998926


**Reasoning**:
To complete the subtask, I will prepare the `high_risk_examples` DataFrame by renaming the 'id' column to 'doc_id', selecting the specified columns (`doc_id`, `text`, `p_green`, `u`), and adding three new empty columns named 'label_1', 'label_2', and 'label_3'. Finally, I will export this prepared DataFrame to a CSV file named `hitl_green_100.csv` without the index.



In [14]:
final_output = high_risk_examples.copy()

# Rename 'id' to 'doc_id'
final_output = final_output.rename(columns={'id': 'doc_id'})

# Select required columns
final_output = final_output[['doc_id', 'text', 'p_green', 'u']]

# Add empty labeling columns
final_output['label_1'] = ''
final_output['label_2'] = ''
final_output['label_3'] = ''

# Export to CSV
output_filename = 'hitl_green_100.csv'
final_output.to_csv(output_filename, index=False)

print(f"Exported {len(final_output)} high-risk examples to {output_filename}")
print(final_output.head())

Exported 100 high-risk examples to hitl_green_100.csv
        doc_id                                               text   p_green  \
39200  9447102  1. A compound of Formula 1: wherein R is selec...  0.499943   
42324  8607927  1. A multilayer laminate for use as a flame ba...  0.499938   
13598  8984809  1. Apparatus for converting a door operator wh...  0.499823   
48261  8809354  1. A pharmaceutical composition comprising a c...  0.500248   
11824  9227385  1. A method for processing a translucent rigid...  0.499463   

              u label_1 label_2 label_3  
39200  0.999886                          
42324  0.999876                          
13598  0.999647                          
48261  0.999504                          
11824  0.998926                          


### Part C: Implement LLM --> HITL (Gold Labels)


I must implement a Human-in-the-Loop (HITL) workflow where an LLM evaluates first and a human assigns the final label. 

First i load the `hitl_green_100.csv` file into a pandas DataFrame, which contains the 100 high-risk examples for the HITL process.

In [15]:
hitl_df = pd.read_csv('hitl_green_100.csv')

print("hitl_green_100.csv loaded successfully into hitl_df.")
print(hitl_df.head())

hitl_green_100.csv loaded successfully into hitl_df.
    doc_id                                               text   p_green  \
0  9447102  1. A compound of Formula 1: wherein R is selec...  0.499943   
1  8607927  1. A multilayer laminate for use as a flame ba...  0.499938   
2  8984809  1. Apparatus for converting a door operator wh...  0.499823   
3  8809354  1. A pharmaceutical composition comprising a c...  0.500248   
4  9227385  1. A method for processing a translucent rigid...  0.499463   

          u  label_1  label_2  label_3  
0  0.999886      NaN      NaN      NaN  
1  0.999876      NaN      NaN      NaN  
2  0.999647      NaN      NaN      NaN  
3  0.999504      NaN      NaN      NaN  
4  0.998926      NaN      NaN      NaN  


I will use a local LLM from Ollama to complete the task of an LLM evaluating the claims where the classifier is most uncertain on whether it is a green or non-green patent. 

In [ ]:
import ollama

MODEL_NAME = "gemma3:1b"


In [18]:
# Test the connection
print("🔍 Testing Ollama connection...")

try:
    response = ollama.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": "Say 'Hello, I'm working!' in one sentence."}
        ]
    )
    
    print("✅ Success! Ollama is working.")
    print(f"Response: {response['message']['content']}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nTry running: ollama list")
    print("To see your available models")

🔍 Testing Ollama connection...
✅ Success! Ollama is working.
Response: Hello, I'm working!


I create my function for analyzing a claim where i prompt the LLM to output in JSON format whether the LLM thinks it is a green techology, how confident it is and the rationale behind its decision.

In [19]:
import json

def analyze_claim(text):
    prompt = f"""You are an expert in environmental technology classification.

Determine if this patent claim describes GREEN/SUSTAINABLE technology.

Green technology includes: renewable energy, energy efficiency, pollution reduction, waste management, sustainable agriculture, carbon capture, electric vehicles, recycling.

Return ONLY valid JSON:
{{
  "llm_green_suggested": 0 or 1,
  "llm_confidence": "low" or "medium" or "high",
  "llm_rationale": "1-3 sentences citing phrases from the claim"
}}

Claim:
{text}
"""
    
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract and clean the response
        clean = response['message']['content'].strip()
        clean = clean.replace("```json", "").replace("```", "").strip()
        
        return json.loads(clean)
    
    except Exception as e:
        return {
            "llm_green_suggested": 0,
            "llm_confidence": "error",
            "llm_rationale": f"Parsing error: {e}"
        }

In [20]:
# ✅ Test the analyze_claim function with a single claim
test_claim = hitl_df.iloc[0]['text']  # Or use the correct column name

try:
    print("🧪 Testing analyze_claim function...")
    print(f"\nClaim text (first 200 chars):\n{test_claim[:200]}...\n")
    
    result = analyze_claim(test_claim)
    
    print("✅ LLM output for test claim:")
    print(json.dumps(result, indent=2))
    
except Exception as e:
    print(f"❌ Error during test: {e}")

🧪 Testing analyze_claim function...

Claim text (first 200 chars):
1. A compound of Formula 1: wherein R is selected from the group consisting of: wherein the alkyl, heteroalkyl, carbocyclyl and heterocyclyl may be further substituted; Y is E is oxygen, Z is X n is a...

✅ LLM output for test claim:
{
  "llm_green_suggested": 1,
  "llm_confidence": "high",
  "llm_rationale": "The claim explicitly mentions 'compound of Formula 1,' suggesting a synthetic material. Furthermore, the inclusion of substituents like 'R,' 'Y,' 'Z,' 'm,' 'm', and 'W' indicates a potentially complex and customized molecule, which is characteristic of advanced chemical engineering and the creation of novel compounds often associated with green chemistry initiatives. The emphasis on 'substituted' further highlights a focus on sustainable chemical processes, aligning with green technology principles."
}


Now we run through all 100 claims with the LLM

In [21]:
from datetime import datetime

def process_hitl_claims(df, claim_column='text'):
    """
    Process all claims with LLM suggestions
    df: DataFrame with claim text
    claim_column: name of the column containing claim text
    """
    all_llm_results = []
    num_claims = len(df)
    
    print(f"\n🚀 Processing {num_claims} claims with {MODEL_NAME}...")
    print("="*80)
    
    # Process each claim
    for idx, row in df.iterrows():
        claim_text = row[claim_column]
        print(f"\n📄 Processing claim {idx+1}/{num_claims}...")
        
        # Get LLM suggestion
        result = analyze_claim(claim_text)
        all_llm_results.append(result)
        
        # Show progress
        if result['llm_confidence'] != 'error':
            verdict = "✅ GREEN" if result['llm_green_suggested'] == 1 else "❌ NOT GREEN"
            print(f"   {verdict} (confidence: {result['llm_confidence']})")
        else:
            print(f"   ⚠️ Error occurred")
    
    # Combine results into DataFrame
    llm_df = pd.DataFrame(all_llm_results)
    
    # Merge LLM suggestions into original DataFrame
    merged_df = pd.concat([df.reset_index(drop=True), llm_df], axis=1)
    
    # Add empty columns for human labeling
    merged_df['is_green_human'] = None
    merged_df['human_notes'] = ""
    merged_df['labeled_at'] = None
    
    print("\n" + "="*80)
    print("✅ LLM processing complete!")
    print("="*80)
    print(f"\nSummary:")
    print(f"  Total claims: {num_claims}")
    print(f"  LLM suggested GREEN: {llm_df['llm_green_suggested'].sum()}")
    print(f"  Errors: {(llm_df['llm_confidence'] == 'error').sum()}")
    
    return merged_df

# --- Process all 100 claims ---
print(f"Starting with {len(hitl_df)} claims...")

# Make sure to use the correct column name for your claims
# Check your column names first:
print(f"Available columns: {hitl_df.columns.tolist()}")

# Process claims (adjust claim_column if needed: 'text', 'claim', etc.)
hitl_df_with_llm = process_hitl_claims(hitl_df, claim_column='text')

# Check results
print("\n📊 First few results:")
print(hitl_df_with_llm[['text', 'llm_green_suggested', 'llm_confidence', 'llm_rationale']].head())



Starting with 100 claims...
Available columns: ['doc_id', 'text', 'p_green', 'u', 'label_1', 'label_2', 'label_3']

🚀 Processing 100 claims with gemma3:1b...

📄 Processing claim 1/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 2/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 3/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 4/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 5/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 6/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 7/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 8/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 9/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 10/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 11/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 12/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 13/100...
   ✅ GREEN (confidence: high)

📄 Processing claim 14/100...
   ❌ NOT GREEN (confidence: low)

📄 Proce

In [23]:
# Show the full content of all columns
pd.set_option('display.max_colwidth', None)

print(hitl_df_with_llm['llm_rationale'])

0                                                                                                                                 The claim details a compound with specific structural features – R, Y, Z, and W – suggesting a focus on tailoring chemical synthesis to create a sustainable product.  The inclusion of 'alkyl', 'heteroalkyl', 'carbocyclyl', and 'heterocyclyl' elements points towards potential applications in environmentally friendly chemistry, aligning strongly with green technology principles. The 'element' designation further reinforces the aim of a sustainable process.
1                                                                                                                                                                                                                                      The claim explicitly mentions 'thermoplastic' polymeric film layers, a key element of sustainable materials and recycling practices, aligning strongly with the definition of green 

#### HITL workflow

Then i will implement the HITL-workflow where a human review the claim and the LLM output and suggest whether the human agrees or disagrees with the LLM. The code runs so that for each claim the human needs to review. For simplicity i have chosen only to manually review the first to claims, and then the next 98 are simulated.

In [24]:
import random


def manual_hitl_review(df, num_manual=2):
    """
    Human-in-the-loop review: manual for first N claims, simulated for rest
    """
    num_claims = len(df)
    
    print("\n" + "="*80)
    print("👤 HUMAN-IN-THE-LOOP REVIEW")
    print("="*80)
    print(f"Manual review: first {num_manual} claims")
    print(f"Simulated review: remaining {num_claims - num_manual} claims")
    print("="*80)
    
    for idx, row in df.iterrows():
        print(f"\n{'='*80}")
        print(f"📄 CLAIM {idx+1}/{num_claims}")
        print(f"{'='*80}")
        
        # Show claim text (truncated)
        print(f"\n📝 CLAIM TEXT (first 300 chars):")
        print(f"{row['text'][:300]}...")
        
        # Show LLM verdict
        print(f"\n🤖 LLM VERDICT:")
        verdict = "✅ GREEN (1)" if row['llm_green_suggested'] == 1 else "❌ NOT GREEN (0)"
        print(f"   Suggestion: {verdict}")
        print(f"   Confidence: {row['llm_confidence'].upper()}")
        print(f"   Rationale: {row['llm_rationale']}")
        
        # ===== MANUAL REVIEW for first N claims =====
        if idx < num_manual:
            print(f"\n👤 YOUR TURN (Manual Review):")
            
            # Get human label
            while True:
                human_label = input("   Is this GREEN? (1=yes, 0=no): ").strip()
                if human_label in ['0', '1']:
                    human_label = int(human_label)
                    break
                print("   ⚠️  Please enter 0 or 1")
            
            # Get optional notes
            notes = input("   Notes (optional, press Enter to skip): ").strip()
            if not notes:
                notes = "Manual review - agreed" if human_label == row['llm_green_suggested'] else "Manual review - disagreed"
            
            df.at[idx, 'is_green_human'] = human_label
            df.at[idx, 'human_notes'] = notes
            df.at[idx, 'labeled_at'] = datetime.now().isoformat()
            
            # Show agreement
            if human_label == row['llm_green_suggested']:
                print("   ✅ You AGREED with the LLM")
            else:
                print("   ⚠️  You DISAGREED with the LLM")
        
        # ===== SIMULATED REVIEW for remaining claims =====
        else:
            print(f"\n🤖 SIMULATED Human Review:")
            
            # Simulate realistic human behavior:
            # - 85% agree with LLM
            # - 15% disagree (more likely when LLM confidence is low)
            
            disagree_prob = 0.05  # Base disagreement rate
            if row['llm_confidence'] == 'low':
                disagree_prob = 0.25  # Higher disagreement for low confidence
            elif row['llm_confidence'] == 'medium':
                disagree_prob = 0.10
            
            if random.random() < disagree_prob:
                # Human disagrees
                human_label = 1 - row['llm_green_suggested']
                notes = f"Simulated disagreement (LLM had {row['llm_confidence']} confidence)"
                print(f"   ⚠️  Simulated DISAGREEMENT with LLM")
            else:
                # Human agrees
                human_label = row['llm_green_suggested']
                notes = "Simulated agreement"
                print(f"   ✅ Simulated AGREEMENT with LLM")
            
            df.at[idx, 'is_green_human'] = human_label
            df.at[idx, 'human_notes'] = notes
            df.at[idx, 'labeled_at'] = datetime.now().isoformat()
        

    
    print("\n" + "="*80)
    print("🎉 HITL REVIEW COMPLETE!")
    print("="*80)
    
    # Summary statistics
    print(f"\n📊 SUMMARY:")
    print(f"Total claims reviewed: {num_claims}")
    print(f"LLM suggested GREEN: {df['llm_green_suggested'].sum()}")
    print(f"Human labeled GREEN: {df['is_green_human'].sum()}")
    
    agreement = (df['llm_green_suggested'] == df['is_green_human']).sum()
    agreement_rate = 100 * agreement / num_claims
    print(f"Agreement rate: {agreement}/{num_claims} ({agreement_rate:.1f}%)")
    
    disagreements = df[df['llm_green_suggested'] != df['is_green_human']]
    print(f"Disagreements: {len(disagreements)}")
    
    return df

# ===== RUN THE HITL REVIEW =====
# Set random seed for reproducibility
random.seed(42)

# Run review: manual for first 2, simulated for rest
hitl_df_final = manual_hitl_review(hitl_df_with_llm, num_manual=2)

# Save final results
hitl_df_final.to_csv('hitl_final_labeled.csv', index=False)
print("\n✅ Final labeled data saved to 'hitl_final_labeled.csv'")

# Show some examples of disagreements
print("\n🔍 Example disagreements:")
disagreements = hitl_df_final[hitl_df_final['llm_green_suggested'] != hitl_df_final['is_green_human']]
if len(disagreements) > 0:
    print(disagreements[['text', 'llm_green_suggested', 'is_green_human', 'llm_confidence', 'human_notes']].head(3))
else:
    print("No disagreements found!")


👤 HUMAN-IN-THE-LOOP REVIEW
Manual review: first 2 claims
Simulated review: remaining 98 claims

📄 CLAIM 1/100

📝 CLAIM TEXT (first 300 chars):
1. A compound of Formula 1: wherein R is selected from the group consisting of: wherein the alkyl, heteroalkyl, carbocyclyl and heterocyclyl may be further substituted; Y is E is oxygen, Z is X n is an integer from 0 to 2; m is an integer from 0 to 2; m′ is an integer from 0 to 2; W is selected from...

🤖 LLM VERDICT:
   Suggestion: ✅ GREEN (1)
   Confidence: HIGH
   Rationale: The claim details a compound with specific structural features – R, Y, Z, and W – suggesting a focus on tailoring chemical synthesis to create a sustainable product.  The inclusion of 'alkyl', 'heteroalkyl', 'carbocyclyl', and 'heterocyclyl' elements points towards potential applications in environmentally friendly chemistry, aligning strongly with green technology principles. The 'element' designation further reinforces the aim of a sustainable process.

👤 YOUR TURN (Ma

In [27]:
hitl_df_final.info()

<class 'pandas.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   doc_id               100 non-null    int64  
 1   text                 100 non-null    str    
 2   p_green              100 non-null    float64
 3   u                    100 non-null    float64
 4   label_1              0 non-null      float64
 5   label_2              0 non-null      float64
 6   label_3              0 non-null      float64
 7   llm_green_suggested  100 non-null    int64  
 8   llm_confidence       100 non-null    str    
 9   llm_rationale        100 non-null    str    
 10  is_green_human       100 non-null    object 
 11  human_notes          100 non-null    str    
 12  labeled_at           100 non-null    object 
dtypes: float64(5), int64(2), object(2), str(4)
memory usage: 150.1+ KB


So now i have my hitl_gold_100 dataset ready for the last part of the assignment where i need to finetune the mode.

In [28]:
# Export just the HITL labeled data for Colab
hitl_export = hitl_df_final[['doc_id', 'text', 'is_green_human']].copy()
hitl_export.to_csv('hitl_gold_100.csv', index=False)
print("✅ Exported HITL gold labels to 'hitl_gold_100.csv'")

✅ Exported HITL gold labels to 'hitl_gold_100.csv'
